In [ ]:
from src.Common import parse_cmd_args, print_b
from src.datasets.text_datasets.RestaurantDataset import RestaurantDataset
from src.datasets.text_datasets.AmazonDataset import AmazonDataset
from src.datasets.text_datasets.POIDataset import POIDataset

from src.models.text_models.ATT2ITM import ATT2ITM

from bokeh.plotting import ColumnDataSource, figure, output_file, save, show
from bokeh.models import LinearColorMapper, Span, transforms
from sklearn.manifold import TSNE

import tensorflow as tf
import pandas as pd
import numpy as np
import nvgpu
import json

from bokeh.layouts import gridplot
from bokeh.resources import INLINE
from bokeh.io import output_notebook, export_svg
output_notebook(INLINE)

In [3]:
gpu = int(np.argmin(list(map(lambda x: x["mem_used_percent"], nvgpu.gpu_info())))) 

model = "ATT2ITM"
dataset = "restaurants".lower().replace(" ", "") 
subset = "newyorkcity".lower().replace(" ", "")

best_model = pd.read_csv("models/best_models.csv")
best_model = best_model.loc[(best_model.dataset == dataset) & (best_model.subset == subset) & (best_model.model == model)]["model_md5"].values[0]
model_path = f"models/{model}/{dataset}/{subset}/{best_model}"
with open(f'{model_path}/cfg.json') as f: model_config = json.load(f)
dts_cfg = model_config["dataset_config"]
with open(f'{model_path}/cfg.json') as f: model_config = json.load(f)
mdl_cfg = {"model": model_config["model"], "session": {"gpu": gpu, "mixed_precision": False, "in_md5": False}}

print_b(f"Loading best model: {best_model}")

if dataset == "restaurants":
    # text_dataset = RestaurantDataset(dts_cfg, load=["TRAIN_DEV", "TEXT_TOKENIZER", "TEXT_SEQUENCES", "WORD_INDEX", "VOCAB_SIZE", "MAX_LEN_PADDING", "N_ITEMS", "FEATURES_NAME", "BOW_SEQUENCES"])
    text_dataset = RestaurantDataset(dts_cfg)
elif dataset == "pois":
    text_dataset = POIDataset(dts_cfg)
elif dataset == "amazon":
    text_dataset = AmazonDataset(dts_cfg)
else:
    raise ValueError

model = ATT2ITM(mdl_cfg, text_dataset)
model.train(dev=True, save_model=True) # Cargar el modelo

Loading best model: 8b74c00371d98f236fb265dd46b234c4


[nltk_data] Downloading package stopwords to /home/pperez/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
wrd_embs = tf.keras.models.Model(inputs=[model.MODEL.input[0]], outputs=[model.MODEL.get_layer("word_emb").output])
rst_embs = tf.keras.models.Model(inputs=[model.MODEL.input[1]], outputs=[model.MODEL.get_layer("rest_emb").output])

rst_embs = rst_embs.predict([list(range(model.DATASET.DATA["N_ITEMS"]))], verbose=0).squeeze()
rest_names = model.DATASET.DATA["TRAIN_DEV"][["id_item", "name"]].sort_values("id_item").drop_duplicates().name.values.tolist()

word_names = np.array(["UNK"]+list(model.DATASET.DATA["WORD_INDEX"].keys()))
wrd_embs = wrd_embs.predict(list(range(model.DATASET.DATA["VOCAB_SIZE"])), verbose=0).squeeze()

In [6]:
tsne_r = TSNE(n_components=2, learning_rate="auto", init="pca", metric="cosine")
tsne_w = TSNE(n_components=2, learning_rate="auto", init="pca", metric="cosine")
rst_tsne = tsne_r.fit_transform(rst_embs)
wrd_tsne = tsne_w.fit_transform(wrd_embs)

In [181]:
from scipy.spatial.distance import cdist
import bokeh

plot_size = 200

# Colorear los puntos en función de la distancia coseno a un restaurante dado.
words = ["i", "fresh", "cheap", "hotdog", "tacos", "pizza", "pasta", "burger"]
words = ["cheese", "beer", "i", "pizza"]
plots = []

for word in words:
    word_id = np.argwhere(word_names==word)[0][0]
    word_colors = cdist([wrd_embs[word_id]], wrd_embs, metric="cosine")[0]
    word_colors = ((word_colors-word_colors.min())/(word_colors.max()-word_colors.min()))
    word_alpha = 1-((word_colors-word_colors.min())/(word_colors.max()-word_colors.min()))

    print(word_names[word_id], word_names[np.argsort(word_colors)][1:4])

    # Graph ------------

    source_w = ColumnDataSource(data=dict(x=wrd_tsne[:, 0], y=wrd_tsne[:, 1], desc=word_names, col=word_colors, alpha=word_alpha))

    TOOLTIPS = [("Name", "@desc"), ("Color", "@col")]
    lc = LinearColorMapper(palette=bokeh.palettes.OrRd[9], low=min(word_colors), high=max(word_colors))
    p = figure(width=plot_size, height=plot_size, tooltips=TOOLTIPS, title=f"{word_names[word_id]}", output_backend="svg")
    p.scatter('x', 'y', size=5, source=source_w, line_color=None, fill_color={"field": "col", "transform": lc}, fill_alpha = "alpha")
    p.axis.visible = False
    plots.append(p)

# put all the plots in an HBox
max_cols = 4
plots = [plots[i:i+max_cols] for i in range(0, len(plots), max_cols)]
p = gridplot(plots)
show(p)

# export_svg(p, filename=f"tsne_words.svg")

cheese ['fixin' 'cheesy' 'macaroni']
beer ['ale' 'pint' 'draft']
i ['my' 'if' 'quite']
pizza ['margherita' 'pizzeria' 'pepperoni']


['tsne_words.svg']

In [180]:
word_names[np.argsort(word_colors)][1:4]

array(['margherita', 'pizzeria', 'pepperoni'], dtype='<U22')

In [191]:
from scipy.spatial.distance import cdist
import bokeh

plot_size = 200

# Colorear los puntos en función de la distancia coseno a un restaurante dado.
items = [865, 834, 338, 613]
plots = []
for item_id in items:
    item_colors = cdist([rst_embs[item_id]], rst_embs, metric="cosine")[0]
    item_colors = ((item_colors-item_colors.min())/(item_colors.max()-item_colors.min()))
    item_alpha = 1-((item_colors-item_colors.min())/(item_colors.max()-item_colors.min()))

    print(rest_names[item_id], pd.unique(np.array(rest_names)[np.argsort(item_colors)])[1:5])


    # Graph ------------
    source_r = ColumnDataSource(data=dict(id=list(range(len(rest_names))), x=rst_tsne[:, 0], y=rst_tsne[:, 1], desc=rest_names, col=item_colors, alpha=item_alpha))

    TOOLTIPS = [("Name", "[@id] @desc"), ("Color", "@col")]
    lc = LinearColorMapper(palette=bokeh.palettes.OrRd[9], low=min(item_colors), high=max(item_colors))
    p = figure(width=plot_size, height=plot_size, tooltips=TOOLTIPS, title=f"Dist. to \"{rest_names[item_id]}\".", output_backend="svg")
    p.scatter('x', 'y', size=5, source=source_r, line_color=None, fill_color={"field": "col", "transform": lc}, fill_alpha = "alpha")
    p.axis.visible = False
    plots.append(p)

# put all the plots in an HBox
max_cols = 4
plots = [plots[i:i+max_cols] for i in range(0, len(plots), max_cols)]
p = gridplot(plots)

show(p)
# export_svg(p, filename=f"tsne_items.svg")                                             

99 Cent Fresh Pizza ['99 Cents Fresh Pizza' '2 Bros Pizza' "Mamoun's Falafel" 'Gotham Pizza']
Chipotle Mexican Grill ['Dos Toros Taqueria' 'Pret' 'Pret A Manger' 'Roast Kitchen']
Bella Napoli ['Luna Restaurant & Pizza' 'Casa Bella Restaurant' 'La Bella Vita'
 'Trattoria Casa di Isacco']
Umi Sushi ["Luanne's Wild Ginger" 'Tomoe Sushi' 'Hatsuhana Restaurant'
 "Ed's Lobster Bar"]


In [189]:
pd.unique(np.array(rest_names)[np.argsort(item_colors)])[1:4]

array(["Luanne's Wild Ginger", 'Tomoe Sushi', 'Hatsuhana Restaurant'],
      dtype='<U65')

In [151]:
[(idr, n) for idr, n in enumerate(rest_names) if "Sush" in n]

[(11, 'Tomoe Sushi'),
 (82, 'Sushi of Gari'),
 (96, 'Sushi Seki'),
 (179, 'Kodama Sushi'),
 (201, 'Sushi Yasuda'),
 (322, 'Blue Ribbon Sushi'),
 (613, 'Umi Sushi'),
 (691, 'Sushi Damo Damo'),
 (740, 'Blue Ribbon Sushi Bar & Grill'),
 (949, 'Haru Sushi'),
 (961, 'Sushi of Gari 46'),
 (1257, 'Sushi Yasaka'),
 (1351, 'Blue Ribbon Sushi Izakaya'),
 (1423, 'Beyond Sushi'),
 (1648, 'Sushi Nakazawa'),
 (1675, 'Wasabi Sushi & Bento'),
 (1814, 'Beyond Sushi')]